The Basic Gravity Model
========================

This example is borrowed from [A Primer in Econometric Theory](https://www.amazon.com/Primer-Econometric-Theory-MIT-Press/dp/0262034905/ref=sr_1_1?ie=UTF8&qid=1466033301&sr=8-1&keywords=a+primer+in+econometric+theory)

The gravity model relates international trade flows
between country $i$ and country $j$ via the equation 
    
$$
    T_{ij} = 
    \frac{\lambda G_i^{\alpha} G_j^\beta}
    {D_{ij}^{\gamma}}
$$

Here $T_{ij}$ is exports from country $i$ to country $j$, $\lambda$ is a constant term, $G_i$ and $G_j$ are GDP in country $i$  and $j$ respectively, and $D_{ij}$ is distance between them.  Taking logs gives the linear model

$$
    \ln T_{ij} 
        = \ln \lambda 
        +  \alpha \ln G_i + \beta \ln G_j - \gamma \ln D_{ij}
        + \epsilon
$$

Here our aim is to give a simple example of linear regression based around this example, using Python's [statsmodels](http://statsmodels.sourceforge.net/) and [pandas](http://pandas.pydata.org/).

Data
-----
Trade data is sourced from: http://atlas.media.mit.edu/en/resources/data and uses the SITC rev 2. product level trade dataset. This product level data is aggregated to form a bilateral trade dataset at the country level.

Distance and Geography is sourced from CEPII: http://www.cepii.fr/CEPII/en/bdd_modele/presentation.asp?id=6

GDP and Population statistics are from the World Development Indicators: http://data.worldbank.org/data-catalog/world-development-indicators

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from numpy import log
import statsmodels.formula.api as smf

### Preparation of Data using Pandas

We will start from a large dataset of **full bilateral trade** data from 1962 to 2014 and extract the year 2013 and use it to estimate the most basic gravity model using ordinary least squares. 

This file contains product level data and is ~4GB of raw data

In [2]:
#-Trade Data File-#
trade = "./data/year_origin_destination_sitc_rev2.tsv"
#Check the Structure of the Trade Data-#
df = pd.read_csv(trade, sep="\t", nrows=5)       #Look at first five rows
df

,year,origin,dest,sitc,export_val,import_val
0,1962,ago,civ,6785,0.0,3000.0
1,1962,ago,civ,2654,158000.0,0.0
2,1962,ago,civ,6651,1000.0,0.0
3,1962,ago,cod,2483,0.0,1000.0
4,1962,ago,cod,1123,0.0,2000.0


For this example we will want to **transform** this to **country level data** by aggregating over the products to obtain export / import data from country $i$ to country $j$. We will then merge this trade data with other data such as the distance between country $i$ and country $j$ and GDP data from the World Development Indicators

First lets check the years in the **RAW** dataset

In [3]:
#-Check years and number of observations in the raw data-#
obs = 0
years = set()
iter_csv = pd.read_csv(trade, sep='\t', iterator=True, chunksize=1000000, usecols=["year"])
for chunk in iter_csv:
    obs += len(chunk)
    years = years.union(set(chunk.year.unique()))

In [4]:
print(years)

{1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014}


In [5]:
print(obs)

117815053


Let us split up the raw data and focus on the cross-section for the year 2013

In [6]:
#Note: Use a Pre-Computed Intermediate File Saved for Presentation

#-Separate 2013 Year Data-#
# iter_csv = pd.read_csv(trade, sep='\t', iterator=True, chunksize=1000000)
# data = pd.concat([chunk[chunk['year'] == 2013] for chunk in iter_csv])
# data.to_csv("data/year_origin_destination_sitc_rev2_2013.tsv", sep="\t")   

To save time -- we will use a precompiled file for the year 2013

In [7]:
#-Year: 2013-#
data = pd.read_csv("data/year_origin_destination_sitc_rev2_2013.tsv", sep="\t", index_col=0)
data.head()

,year,origin,dest,sitc,export_val,import_val
426778,2013,bdi,bdi,7283,NaN,188547.0
426779,2013,bdi,bdi,7810,NaN,7443.0
426780,2013,bdi,ben,9310,3003.0,86340.0
426781,2013,bdi,bfa,8212,NaN,873.0
426782,2013,bdi,bfa,6534,NaN,10953.0


In [8]:
#-Working with 3,663,399 Observations-#
data.shape

(3663399, 6)

Now we can transform the data by adding all of the export values for each 'year', 'origin', and 'destination' across the product dimension. 

For this example we will focus on importer reports.

In [9]:
data = data[['year','origin','dest','import_val']].groupby(["year","origin","dest"]).sum()
data.head()

import_val
year origin dest            
2013 abw    bel     774353.0
            bhs    4712537.0
            che   17812626.0
            chn   25319168.0
            col   22160086.0

In [10]:
#-Reset the Index, Rename the Columns, and Harmonise the Country Codes to Upper Case-#
data = data.reset_index()
data.rename(columns={'import_val':'value','origin':'eiso3c','dest':'iiso3c'}, inplace=True)
data.eiso3c = data.eiso3c.apply(lambda x: x.upper())
data.iiso3c = data.iiso3c.apply(lambda x: x.upper())

In [11]:
data.head()

,year,eiso3c,iiso3c,value
0,2013,ABW,BEL,774353.0
1,2013,ABW,BHS,4712537.0
2,2013,ABW,CHE,17812626.0
3,2013,ABW,CHN,25319168.0
4,2013,ABW,COL,22160086.0


### Add in other data for distance, exporter and importer gdp

Add in **Bilateral** attribute such as distance

In [12]:
#-CEPII Bilateral Data-#
dist = pd.read_excel('data/dist_cepii.xls')
dist.head()

,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces
0,ABW,ABW,0,0,0,0,0,0,0,0,5.225315,5.225315,25.0935,23.0472
1,ABW,AFG,0,0,0,0,0,0,0,0,13257.810000,13257.810000,13168.2,13166.4
2,ABW,AGO,0,0,0,0,0,0,0,0,9516.913000,9516.913000,9587.32,9584.19
3,ABW,AIA,0,0,1,0,0,0,0,0,983.268200,983.268200,976.897,976.892
4,ABW,ALB,0,0,0,0,0,0,0,0,9091.742000,9091.742000,9091.58,9091.47


In [13]:
#-Select the Data we are Interested in-#
bilat_attrs = ["iso_o","iso_d","dist"]
dist = dist[bilat_attrs].drop_duplicates()     #Ensure Uniqueness for Merging
dist.head()

,iso_o,iso_d,dist
0,ABW,ABW,5.225315
1,ABW,AFG,13257.810000
2,ABW,AGO,9516.913000
3,ABW,AIA,983.268200
4,ABW,ALB,9091.742000


In [14]:
data.head()

,year,eiso3c,iiso3c,value
0,2013,ABW,BEL,774353.0
1,2013,ABW,BHS,4712537.0
2,2013,ABW,CHE,17812626.0
3,2013,ABW,CHN,25319168.0
4,2013,ABW,COL,22160086.0


In [15]:
#-Merge the Data-#
data = data.merge(dist, left_on=["eiso3c","iiso3c"], right_on=["iso_o","iso_d"], how="inner")

In [16]:
#-Cleanup Unnecessary Items-#
for item in ["iso_o","iso_d"]:
	del data[item]

In [17]:
data.head()

,year,eiso3c,iiso3c,value,dist
0,2013,ABW,BEL,774353.0,7847.070
1,2013,ABW,BHS,4712537.0,1588.515
2,2013,ABW,CHE,17812626.0,8056.332
3,2013,ABW,CHN,25319168.0,14155.350
4,2013,ABW,COL,22160086.0,1036.634


In [18]:
#-Check the Merge is Correct-#
dist[(dist.iso_o == "ABW") & (dist.iso_d == "BEL")]

,iso_o,iso_d,dist
15,ABW,BEL,7847.07


#### Add in Exporter and Importer GDP from World Bank WDI

In [19]:
#-Add World Development Indicators GDP Data-#
gdp = pd.read_csv("data/wdi_gdp.csv")
gdp.head(n=1)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Afghanistan,AFG,GDP at market prices (constant 2005 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,6.623602e+09,7.533699e+09,7.805769e+09,9.446592e+09,1.024325e+10,1.086949e+10,1.243847e+10,1.268216e+10,1.284862e+10,NaN


In [20]:
#-Select Data of Interest and Rename Data to descriptive name-#
gdp2013 = gdp[['Country Code', '2013']].rename(columns={'2013': 'gdp'})

In [21]:
gdp2013.head()

,Country Code,gdp
0,AFG,1.268216e+10
1,ALB,1.104057e+10
2,DZA,1.271905e+11
3,ASM,NaN
4,ADO,2.644100e+09


In [22]:
#-Merge Exporter GDP Information-#
data = data.merge(gdp2013, how='inner', left_on=['eiso3c'], right_on=['Country Code'])
del data['Country Code']
data.rename(columns={'gdp' : 'egdp'}, inplace=True)

In [23]:
data.dropna().head()

,year,eiso3c,iiso3c,value,dist,egdp
19,2013,AFG,CHN,136251991.0,4180.438,1.268216e+10
20,2013,AFG,DEU,170779239.0,5226.514,1.268216e+10
21,2013,AFG,GBR,290986.0,5718.078,1.268216e+10
22,2013,AFG,IND,43729538.0,1003.893,1.268216e+10
23,2013,AFG,IRN,715151592.0,1620.826,1.268216e+10


In [24]:
#-Merge Exporter GDP Information-#
data = data.merge(gdp2013, how='inner', left_on=['iiso3c'], right_on=['Country Code'])
del data['Country Code']
data.rename(columns={'gdp' : 'igdp'}, inplace=True)

In [25]:
data.head()

,year,eiso3c,iiso3c,value,dist,egdp,igdp
0,2013,ABW,BEL,774353.0,7847.070,NaN,4.232125e+11
1,2013,ALB,BEL,22101708.0,1589.107,1.104057e+10,4.232125e+11
2,2013,ARG,BEL,412813198.0,11326.760,3.278999e+11,4.232125e+11
3,2013,ARM,BEL,70188189.0,3299.886,6.861761e+09,4.232125e+11
4,2013,ATG,BEL,902915.0,6881.374,1.030310e+09,4.232125e+11


In [26]:
#-Clean Up by Dropping NaN Values-#
print(data.shape)
data = data.dropna()
print(data.shape)

(22695, 7)
(19713, 7)


### Description of the data

In [27]:
data.describe()

,year,value,dist,egdp,igdp
count,19713.0,1.971300e+04,19713.000000,1.971300e+04,1.971300e+04
mean,2013.0,7.952853e+08,7475.358396,4.820946e+11,3.865739e+11
std,0.0,6.974516e+09,4420.311050,1.556033e+12,1.388119e+12
min,2013.0,1.000000e+00,8.300385,1.203875e+08,2.620733e+07
25%,2013.0,1.053100e+05,3979.619000,1.139811e+10,8.635166e+09
50%,2013.0,3.592025e+06,7069.655000,4.530376e+10,3.158064e+10
75%,2013.0,7.231337e+07,10396.750000,2.530542e+11,2.024215e+11
max,2013.0,4.220679e+11,19812.040000,1.445151e+13,1.445151e+13


The names of all the variables are as follows:

In [28]:
data.columns

Index(['year', 'eiso3c', 'iiso3c', 'value', 'dist', 'egdp', 'igdp'], dtype='object')

Here `egdp` is export country GDP, `igdp` is import country GDP and `dist` is distance.  These are our regressors.  The dependent variable is `value`.

### Estimation 

Estimate the basic gravity model

In [29]:
formula = "log(value) ~ log(egdp) + log(igdp) + log(dist)"
model = smf.ols(formula, data)
result = model.fit(cov_type='HC1')
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:             log(value)   R-squared:                       0.653
Model:                            OLS   Adj. R-squared:                  0.653
Method:                 Least Squares   F-statistic:                 1.203e+04
Date:                Wed, 15 Jun 2016   Prob (F-statistic):               0.00
Time:                        23:17:00   Log-Likelihood:                -47217.
No. Observations:               19713   AIC:                         9.444e+04
Df Residuals:                   19709   BIC:                         9.447e+04
Df Model:                           3                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept    -30.0796      0.394    -76.280      0.0